#Phase 2: Modeling & Hyperparameter Tuning 

In [2]:
!pip install -q -U keras-tuner

In [3]:
%matplotlib inline 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from kerastuner.tuners import Hyperband
from kerastuner import HyperModel
import matplotlib.pyplot as plt #graphs 
import tensorflow as tf 
import keras_tuner as kt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  This is separate from the ipykernel package so we can avoid doing imports until


**Data Augmentation & Test, Train, Validation Split**

In [4]:
#Split validation set by 20%
#Image generator helps creating image aumentation to increase the amount of data we have
#Will implement various rotations and flips to the images to distort
train_gen = ImageDataGenerator(rescale = 1./255, 
                               validation_split=0.20, 
                               rotation_range=45, 
                               width_shift_range=0.2, 
                               height_shift_range=0.2, 
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale = 1./255)


def get_train_set(train_gen, input_size):
  train_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                         class_mode='categorical',
                                         target_size= input_size,
                                         color_mode = 'grayscale',
                                         batch_size= 32, 
                                         shuffle = True, 
                                         subset ='training')
  return train_set

def get_validation_set(train_gen, input_size):
  validation_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                          target_size= input_size,
                                          color_mode = 'grayscale',
                                          class_mode='categorical',
                                          batch_size= 32, 
                                          shuffle = True, 
                                          subset ='validation')
  return validation_set

def get_test_set(test_gen, input_size):
  test_set = test_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/test",
                                         target_size=input_size, 
                                         color_mode = 'grayscale',
                                         class_mode='categorical',
                                         batch_size= 1,
                                         shuffle = True)
  return test_set

In [5]:
#get train, val, test sets 
input_shape = (224,224,1)

train_set = get_train_set(train_gen=train_gen, input_size = (224,224))
val_set = get_validation_set(train_gen=train_gen, input_size = (224,224))
test_set = get_test_set(test_gen=test_gen, input_size = (224,224))

Found 6755 images belonging to 196 classes.
Found 1585 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


##Structural Tuning

In [6]:
#flatten layer = get vector to put in classifier 
# last conv layer followed by a dropout layer to prevent overfitting 
# flatten, fully connected layers 

class Hypermodel(HyperModel):

  def __init__(self, NN_Type, num_dense_layers):

    if (NN_Type != 'GoogleNet' and NN_Type != 'VGG16' and NN_Type != 'ResNet152'):
      raise ValueError("Invalid model type")

    if (num_dense_layers <= 0):
      raise ValueError("Invalid number of layers. You must have a minimum of 1 layer")

    self.NN_Type = NN_Type
    self.num_dense_layers = num_dense_layers

  def build(self, hp):

    if (self.NN_Type == 'GoogleNet'):
      #TODO: have options for pooling (none, avg, global)
      base_model = tf.keras.applications.InceptionV3(include_top = False, weights=None , input_shape=(224, 224, 1))

      #only taking first couple of 
      model = tf.keras.Model(inputs = base_model.input, outputs = base_model.get_layer('mixed7').output)
      x = model.output

    elif (self.NN_Type == 'VGG16'):
      tf.keras.applications.VGG16()
    else: 
      tf.keras.applications.ResNet152()

    #flatten
    x = tf.keras.layers.Flatten()(x)

    #dense
    for i in range(self.num_dense_layers):
      x = tf.keras.layers.Dense(units=hp.Int('units', min_value = 0.0, max_value = 512, step = 32), activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'],default='relu'))(x)

    #dropout
    x = tf.keras.layers.Dropout(rate = hp.Float('dropout', min_value = 0.0, max_value = 0.5, default =  0.1, step = 0.05))(x)

    #classification layer(dense)
    x = tf.keras.layers.Dense(196, activation = 'softmax')(x)

    model = tf.keras.Model(model.input, x)
    model.compile(optimizer= tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3])),loss='categorical_crossentropy',metrics=['accuracy'])

    return model

In [7]:
#don't explore unlikely options 
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8)

#create model checkpoint (save at this point)


hypermodel_Google = Hypermodel(NN_Type = 'GoogleNet', num_dense_layers= 2)
#hypermodel_VGG = Hypermodel(NN_Type = 'VGG16')
#hypermodel_ResNet = Hypermodel(NN_Type = 'GoogleNet')

tuner_GN = Hyperband(hypermodel =hypermodel_Google, objective = 'val_accuracy', max_epochs = 100)
#tuner_VGG = Hyperband(hypermodel=hypermodel_VGG, objective='val_accuracy', max_epochs=100)
#tuner_ResNet = Hyperband(hypermodel=hypermodel_ResNet, objective='val_accuracy', max_epochs=100)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


**GoogleNet Validation**

In [8]:
tuner_GN.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 512, 'step': 32, 'sampling': None}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 0.001, 0.01], 'ordered': True}


In [ ]:
tuner_GN.search(train_set, epochs = 50, validation_data = val_set, callbacks = [stop_early])


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
units             |0                 |448               
dense_activation  |relu              |sigmoid           
dropout           |0.35              |0.1               
learning_rate     |0.01              |0.001             
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |0                 |0                 

Epoch 1/2


In [ ]:
#save best weights
best_hps = tuner_GN.get_best_hyperparameters(num_trials = 1)[0]
best_model_GN = tuner_GN.hypermodel.build(best_hps)
best_model_GN.save("google_net_hyperband") 

**VGG16 Validation**

**ResNet152 Validation**

##Epoch Optimization for Best Models